In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    FewShotChatMessagePromptTemplate,
)

chat = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=180,
    return_messages=True,
)

examples = [
    {"title": "Top Gun", "answer": "🛩️👨‍✈️🔥"},
    {"title": "The Godfather", "answer": "👨‍👨‍👦🔫🍝"},
]

example_template = ChatPromptTemplate.from_messages(
    [("human", "{title}"), ("ai", "{answer}")]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_template, examples=examples
)

example_prompt

FewShotChatMessagePromptTemplate(examples=[{'title': 'Top Gun', 'answer': '🛩️👨\u200d✈️🔥'}, {'title': 'The Godfather', 'answer': '👨\u200d👨\u200d👦🔫🍝'}], example_prompt=ChatPromptTemplate(input_variables=['answer', 'title'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['title'], template='{title}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['answer'], template='{answer}'))]))

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You were gave the title of a movie, and you are to reply with"
                " three emojis that represent the movie."
            ),
        ),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | chat


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


invoke_chain("Frozen")

content='❄️👸⛄'


In [ ]:
load_memory({})

[HumanMessage(content='Frozen'), AIMessage(content='❄️👸⛄')]